In [18]:
from sentence_transformers import SentenceTransformer
import torch
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
import faiss

nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [13]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
def divide_document(text, max_words=300):
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]
    fragments = []
    temp = ""

    for sent in sents:
        if len(temp.split()) + len(sent.split()) <= max_words:
            temp += " " + sent
        else:
            fragments.append(temp.strip())
            temp = sent

    if temp:
        fragments.append(temp.strip())

    return fragments

In [9]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-...,catalogada y caricaturizada bajo el rótulo de ...
1,pack\AIBR\2006-Cambios-de-género-y-discriminac...,agradecimientos la autora desea agradecer a la...
2,pack\AIBR\2006-Consecuencias-personales-en-la-...,has carried out the biggest employment adjustm...
3,pack\AIBR\2006-Construcción-de-modelos-de-géne...,tampoco queremos que se entienda identidad de ...
4,pack\AIBR\2006-Contribuciones-feministas-a-pro...,is irrelevant for others con el que teresa del...
...,...,...
1920,pack\Revista-Española-de-Antropología-American...,"cómo citar: burguete cal y mayor, araceli. 202..."
1921,pack\Revista-Española-de-Antropología-American...,project exposed some critical questionings. in...
1922,pack\Revista-Española-de-Antropología-American...,"1. introducción el interés por los quipus, que..."
1923,pack\Revista-Española-de-Antropología-American...,1. introducción ¿cómo se produce la diferencia...


In [ ]:
fragments = []
fragments_ids = []

for i, row in df.iterrows():
    if type(row["Text"]) != str:
        continue
    frags = divide_document(row["Text"])
    fragments += frags
    fragments_ids += [i] * len(frags)

len(fragments), len(fragments_ids)

ignored


(0, 0)

In [11]:
def divide_and_assign_ids(i, text):
    if type(text) != str:
        return [], []

    frags = divide_document(text)
    return frags, [i] * len(frags)

fragments = []
fragments_ids = []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: divide_and_assign_ids(row[0], row[1]["Text"]),
                                df.iterrows()))

    for frags, ids in results:
        if not frags or not ids:
            continue

        fragments.extend(frags)
        fragments_ids.extend(ids)

len(fragments), len(fragments_ids)

(57716, 57716)

In [14]:
with open("fragments.pkl", "wb") as f:
    pickle.dump((fragments, fragments_ids), f)

In [16]:
embeddings = model.encode(fragments, convert_to_tensor=True, device=device)
print(embeddings.shape)

torch.Size([57716, 384])


In [17]:
embeddings_np = embeddings.cpu().numpy()
embeddings_np.shape

(57716, 384)

In [19]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000023FFF5C7D20> >

In [21]:
faiss.write_index(index, "index-sentence-transformers.faiss")

In [49]:
def search(query, top_k=10):
    query_embedding = model.encode([query], convert_to_tensor=True, device=device).cpu().numpy()
    _, indices = index.search(query_embedding, k=top_k*2)

    already_in = set()
    results = []
    for i in indices[0]:
        id = fragments_ids[i]
        if id in already_in:
            continue

        already_in.add(id)
        results.append(id)
        if len(results) == top_k:
            break

    return df.iloc[results]

In [50]:
q = """
En los anales de la historia peruana, la llegada de Beldever marcó un hito significativo. Proveniente de tierras lejanas, este enigmatico personaje arribó a las costas del Perú en el año 1542, durante una época de grandes convulsiones y transformaciones en el territorio andino. Su presencia no solo despertó curiosidad entre los locales, sino que también generó especulaciones sobre sus verdaderas intenciones. Algunos cronistas de la época relatan que Beldever traía consigo conocimientos avanzados y artefactos desconocidos, lo que le granjeó tanto admiración como recelo entre los colonizadores y los pueblos indígenas. Aunque su estancia en el Perú fue breve, su legado perduró en las leyendas y relatos que se transmitieron de generación en generación, convirtiéndolo en una figura misteriosa y fascinante dentro de la historia colonial del país.
"""

search(q)

,Path,Text
1074,pack\Revista-de-Indias\2007-Al-servicio-de-la-...,fugaz de los tres ministros plenipotenciarios ...
681,pack\Anuario-de-Estudios-Americanos\2014-Los-a...,"introducción a finales de diciembre de 1640, l..."
1462,pack\Revista-de-Indias\2023-Sueños-de-un-imper...,terminar con la primera experiencia liberal en...
1451,pack\Revista-de-Indias\2023-La-odisea-amazónic...,introducción confinado en el fuerte de são ped...
1401,pack\Revista-de-Indias\2021-La-Inquisición-no-...,si la única manera de reivindicar un lugar es ...
927,pack\Revista-de-Indias\2001-El-Perú-y-el-mundo...,"de francis drake en 1579, se repetía que la me..."
674,pack\Anuario-de-Estudios-Americanos\2014-Imagi...,"introducción la guerra del pacífico provocó, a..."
510,pack\Anuario-de-Estudios-Americanos\2005-Los-o...,pone este trabajo es que el desarrollo de las ...
1121,pack\Revista-de-Indias\2009-Boletos-sencillos-...,tizos que viajaron a la península ibérica por ...
1092,pack\Revista-de-Indias\2007-Los-caminos-de-la-...,"no sé qué será de mi casa, de mis libros y de ..."
